In [265]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


from collections import defaultdict
from scipy.stats.stats import pearsonr
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist, squareform
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram

In [266]:
df = pd.read_csv('training.csv')

In [267]:
#BINNING NUMERICAL VALUES
#We have to use the sturges' rule but that gives us 17 bin but i think 10 is better
df['VehBCost_bin'] = pd.cut(df.VehBCost.astype(int), 17, right = False) 
df['VehOdo_bin'] = pd.cut(df.VehOdo.astype(int), 17, right = False)
df['WarrantyCost_bin'] = pd.cut(df.WarrantyCost.astype(int), 17, right = False)

In [268]:
#SELECT THE COLUMNS I AM INTERESTED IN
df_new = df[['Auction','VehicleAge','Make','Trim','Color','WheelType','Nationality','Size',
             'VNST','VehBCost_bin', 'VehOdo_bin','WarrantyCost_bin']]

In [269]:
#FIX MISSING VALUES
df_new.isnull().sum()

Auction                0
VehicleAge             0
Make                   0
Trim                1911
Color                  7
WheelType           2577
Nationality            4
Size                   4
VNST                   0
VehBCost_bin           0
VehOdo_bin             0
WarrantyCost_bin       0
dtype: int64

In [270]:
df_new.dropna(subset = ['Color','Nationality','Size'], inplace=True)

C:\Users\bonin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [271]:
df_new.Trim.fillna('Bas', inplace = True, limit=801)

C:\Users\bonin\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [272]:
df_new.Trim.fillna('LS', inplace = True, limit=580)

In [273]:
df_new.Trim.fillna('SE', inplace = True, limit=530)

In [274]:
df_new.WheelType.fillna('Alloy', inplace = True, limit=1377)

In [275]:
df_new.WheelType.fillna('Covers', inplace = True, limit=1200)

In [276]:
df_new.Color.isnull().sum()
df_new.Nationality.isnull().sum()
df_new.Size.isnull().sum()
df_new.Trim.isnull().sum()
df_new.WheelType.isnull().sum()

0

In [277]:
df_new.isnull().sum()

Auction             0
VehicleAge          0
Make                0
Trim                0
Color               0
WheelType           0
Nationality         0
Size                0
VNST                0
VehBCost_bin        0
VehOdo_bin          0
WarrantyCost_bin    0
dtype: int64

In [278]:
df_new.head()

,Auction,VehicleAge,Make,Trim,Color,WheelType,Nationality,Size,VNST,VehBCost_bin,VehOdo_bin,WarrantyCost_bin
0,ADESA,2,KIA,EX,BLUE,Covers,OTHER ASIAN,MEDIUM,FL,"[4293.235, 6439.353)","[57009.471, 63532.529)","[462.0, 875.882)"
1,MANHEIM,3,DODGE,SXT,SILVER,Covers,AMERICAN,MEDIUM,FL,"[2147.118, 4293.235)","[89624.765, 96147.824)","[1289.765, 1703.647)"
2,OTHER,2,DODGE,SE,BLACK,Covers,AMERICAN,MEDIUM,OK,"[6439.353, 8585.471)","[43963.353, 50486.412)","[462.0, 875.882)"
3,OTHER,4,FORD,S,BROWN,Covers,AMERICAN,VAN,AZ,"[4293.235, 6439.353)","[76578.647, 83101.706)","[1289.765, 1703.647)"
4,MANHEIM,4,CHRYSLER,Bas,BLUE,Alloy,AMERICAN,VAN,TX,"[4293.235, 6439.353)","[76578.647, 83101.706)","[1289.765, 1703.647)"


In [279]:
#this code is necessary because if we don't add a string to the algorithm value..
#..when we run the algorithm it doesn't find the attribute values
for i in df_new:
        df_new[i] = df_new[i].astype(str) + "_{}".format(i)

C:\Users\bonin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [280]:
df_new.head()

,Auction,VehicleAge,Make,Trim,Color,WheelType,Nationality,Size,VNST,VehBCost_bin,VehOdo_bin,WarrantyCost_bin
0,ADESA_Auction,2_VehicleAge,KIA_Make,EX_Trim,BLUE_Color,Covers_WheelType,OTHER ASIAN_Nationality,MEDIUM_Size,FL_VNST,"[4293.235, 6439.353)_VehBCost_bin","[57009.471, 63532.529)_VehOdo_bin","[462.0, 875.882)_WarrantyCost_bin"
1,MANHEIM_Auction,3_VehicleAge,DODGE_Make,SXT_Trim,SILVER_Color,Covers_WheelType,AMERICAN_Nationality,MEDIUM_Size,FL_VNST,"[2147.118, 4293.235)_VehBCost_bin","[89624.765, 96147.824)_VehOdo_bin","[1289.765, 1703.647)_WarrantyCost_bin"
2,OTHER_Auction,2_VehicleAge,DODGE_Make,SE_Trim,BLACK_Color,Covers_WheelType,AMERICAN_Nationality,MEDIUM_Size,OK_VNST,"[6439.353, 8585.471)_VehBCost_bin","[43963.353, 50486.412)_VehOdo_bin","[462.0, 875.882)_WarrantyCost_bin"
3,OTHER_Auction,4_VehicleAge,FORD_Make,S_Trim,BROWN_Color,Covers_WheelType,AMERICAN_Nationality,VAN_Size,AZ_VNST,"[4293.235, 6439.353)_VehBCost_bin","[76578.647, 83101.706)_VehOdo_bin","[1289.765, 1703.647)_WarrantyCost_bin"
4,MANHEIM_Auction,4_VehicleAge,CHRYSLER_Make,Bas_Trim,BLUE_Color,Alloy_WheelType,AMERICAN_Nationality,VAN_Size,TX_VNST,"[4293.235, 6439.353)_VehBCost_bin","[76578.647, 83101.706)_VehOdo_bin","[1289.765, 1703.647)_WarrantyCost_bin"


In [281]:
#we have to pass the values of df_new as a list to run in the right way apriori algorithm
baskets = df_new.values.tolist()

In [282]:
#BASKET EXAMPLE
baskets[0]

['ADESA_Auction',
 '2_VehicleAge',
 'KIA_Make',
 'EX_Trim',
 'BLUE_Color',
 'Covers_WheelType',
 'OTHER ASIAN_Nationality',
 'MEDIUM_Size',
 'FL_VNST',
 '[4293.235, 6439.353)_VehBCost_bin',
 '[57009.471, 63532.529)_VehOdo_bin',
 '[462.0, 875.882)_WarrantyCost_bin']

In [283]:
from fim import apriori

In [284]:
#This is the explaination of the parameters of apriori
print(apriori.__doc__)

apriori (tracts, target='s', supp=10, zmin=1, zmax=None, report='a',
         eval='x', agg='x', thresh=10, prune=None, algo='b', mode='',
         border=None)
Find frequent item sets with the Apriori algorithm.
tracts  transaction database to mine (mandatory)
        The database must be an iterable of transactions;
        each transaction must be an iterable of items;
        each item must be a hashable object.
        If the database is a dictionary, the transactions are
        the keys, the values their (integer) multiplicities.
target  type of frequent item sets to find     (default: s)
        s/a   sets/all   all     frequent item sets
        c     closed     closed  frequent item sets
        m     maximal    maximal frequent item sets
        g     gens       generators
        r     rules      association rules
supp    minimum support of an item set         (default: 10)
        (positive: percentage, negative: absolute number)
conf    minimum confidence of an assoc. rul

In [285]:
rules = apriori(baskets, supp=10, zmin=2, target='r', conf=80, report='YXlC') 
print(len(rules))
contare = 0
for y in rules:
        print('%s -- %s' % (str(contare), y))
        contare = contare + 1

89
0 -- ('AMERICAN_Nationality', ('[57009.471, 63532.529)_VehOdo_bin',), 83.58543897216273, 10.197858672376874, 0.9704891742639087, 81.11876364858055)
1 -- ('AMERICAN_Nationality', ('BLACK_Color',), 83.58543897216273, 10.391434689507495, 0.9701609480623545, 81.09132871744148)
2 -- ('AMERICAN_Nationality', ('GREY_Color',), 83.58543897216273, 10.764882226980728, 0.9945723110437195, 83.13176320814767)
3 -- ('AMERICAN_Nationality', ('[1703.647, 2117.529)_WarrantyCost_bin',), 83.58543897216273, 10.898501070663812, 1.1529408413280817, 96.36906633134234)
4 -- ('AMERICAN_Nationality', ('6_VehicleAge',), 83.58543897216273, 10.932762312633832, 0.988860516240292, 82.65434033218428)
5 -- ('Alloy_WheelType', ('MEDIUM SUV_Size',), 51.58201284796574, 10.95845824411135, 1.688940721840783, 87.1189620134438)
6 -- ('AMERICAN_Nationality', ('MEDIUM SUV_Size',), 83.58543897216273, 10.95845824411135, 1.0091714664183478, 84.35204001875879)
7 -- ('AMERICAN_Nationality', ('2_VehicleAge',), 83.58543897216273, 1

In [286]:
#RULES EXAMPLE
#INDEX: 
# 0 : 'AMERICAN_Nationality'
# 1 : ('[82449.4, 93538.6)_VehOdo_bin', 'AUTO_Transmission')
# 2 : 83.58515777572207 refers to 'Y' in report='YXlC'  that is relative head(head is 'AMERICAN_Nationality') item support as a percentage
# 3 : 20.27957652379484 refers to 'X' in report='YXlC'  that is X relative body(body is ('[82449.4, 93538.6)_VehOdo_bin', 'AUTO_Transmission')) set  support as a percentage
# 4 : 1.0436783505509324 refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
# 5 : 87.23601959790506 refers to 'C' in report='YXlC'  that is rule confidence as a percentage
rules[4]

('AMERICAN_Nationality',
 ('6_VehicleAge',),
 83.58543897216273,
 10.932762312633832,
 0.988860516240292,
 82.65434033218428)

In [287]:
#Code that filters all the association rules with a certain lift under
count = 0
for r in rules:
    if r[4]>1.3: #r[4] indeed refers to index 4 so as we have seen before index 4 refers to refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
        print('%s -- %s' % (str(count), r))
        count = count + 1
print('Rules with high lift: {}'.format(count))

0 -- ('Alloy_WheelType', ('MEDIUM SUV_Size',), 51.58201284796574, 10.95845824411135, 1.688940721840783, 87.1189620134438)
1 -- ('CHEVROLET_Make', ('LS_Trim', 'AMERICAN_Nationality'), 23.70877944325482, 14.132762312633835, 3.971422315641969, 94.15757575757576)
2 -- ('CHEVROLET_Make', ('LS_Trim',), 23.70877944325482, 15.02355460385439, 3.7359445956723194, 88.57468643101483)
Rules with high lift: 3


In [288]:
#TASKS ON KAGGLE:
# 1) Frequent patterns extraction with different values of support and different types (i.e. frequent, close, maximal), (6 points)
# 2) Discussion of the most interesting frequent patterns and analyze how changes the number of patterns w.r.t. the min_sup parameter (7 points)

In [289]:
# s/a   sets/all   all     frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='a') 
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 610
Min support = 10 
 Itemsets = 133
Min support = 15 
 Itemsets = 50
Min support = 20 
 Itemsets = 20
Min support = 25 
 Itemsets = 11
Min support = 30 
 Itemsets = 7
Min support = 35 
 Itemsets = 3
Min support = 40 
 Itemsets = 2
Min support = 45 
 Itemsets = 1
Min support = 50 
 Itemsets = 0
Min support = 55 
 Itemsets = 0
Min support = 60 
 Itemsets = 0
Min support = 65 
 Itemsets = 0
Min support = 70 
 Itemsets = 0
Min support = 75 
 Itemsets = 0
Min support = 80 
 Itemsets = 0
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [290]:
# c     closed     closed  frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='c')
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 561
Min support = 10 
 Itemsets = 127
Min support = 15 
 Itemsets = 50
Min support = 20 
 Itemsets = 20
Min support = 25 
 Itemsets = 11
Min support = 30 
 Itemsets = 7
Min support = 35 
 Itemsets = 3
Min support = 40 
 Itemsets = 2
Min support = 45 
 Itemsets = 1
Min support = 50 
 Itemsets = 0
Min support = 55 
 Itemsets = 0
Min support = 60 
 Itemsets = 0
Min support = 65 
 Itemsets = 0
Min support = 70 
 Itemsets = 0
Min support = 75 
 Itemsets = 0
Min support = 80 
 Itemsets = 0
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [291]:
#m     maximal    maximal frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='m') 
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 256
Min support = 10 
 Itemsets = 75
Min support = 15 
 Itemsets = 31
Min support = 20 
 Itemsets = 15
Min support = 25 
 Itemsets = 8
Min support = 30 
 Itemsets = 7
Min support = 35 
 Itemsets = 3
Min support = 40 
 Itemsets = 2
Min support = 45 
 Itemsets = 1
Min support = 50 
 Itemsets = 0
Min support = 55 
 Itemsets = 0
Min support = 60 
 Itemsets = 0
Min support = 65 
 Itemsets = 0
Min support = 70 
 Itemsets = 0
Min support = 75 
 Itemsets = 0
Min support = 80 
 Itemsets = 0
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [292]:
#TASK ON KAGGLE:
# 3) Association rules extraction with different values of confidence (6 points)
# 4) Discussion of the most interesting rules and analyze how changes the number of rules w.r.t. the min_conf parameter, histogram of rules' confidence and lift (7 points)

In [293]:
#EXTRACT RULES
for i in range(5,101,5):
    for j in range(5,101,5):
        rules = apriori(baskets, supp=i, zmin=2, target='r', conf=j, report='YXlC') 
        print("Support = {} \nConfidence = {} \nRules = {}".format(i,j,len(rules)))
        #Code that filters all the association rules with a certain lift under
        count = 0
        for r in rules:
            if r[4]>1.4: #r[4] indeed refers to index 4 so as we have seen before index 4 refers to refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
                count = count + 1
        print('Rules with high lift: {}'.format(count))
        print('\n')

Support = 5 
Confidence = 5 
Rules = 30984
Rules with high lift: 5930


Support = 5 
Confidence = 10 
Rules = 21093
Rules with high lift: 4029


Support = 5 
Confidence = 15 
Rules = 14206
Rules with high lift: 3103


Support = 5 
Confidence = 20 
Rules = 8912
Rules with high lift: 2170


Support = 5 
Confidence = 25 
Rules = 5661
Rules with high lift: 1574


Support = 5 
Confidence = 30 
Rules = 4186
Rules with high lift: 1168


Support = 5 
Confidence = 35 
Rules = 3254
Rules with high lift: 831


Support = 5 
Confidence = 40 
Rules = 2594
Rules with high lift: 629


Support = 5 
Confidence = 45 
Rules = 2063
Rules with high lift: 503


Support = 5 
Confidence = 50 
Rules = 1672
Rules with high lift: 418


Support = 5 
Confidence = 55 
Rules = 1313
Rules with high lift: 333


Support = 5 
Confidence = 60 
Rules = 780
Rules with high lift: 255


Support = 5 
Confidence = 65 
Rules = 568
Rules with high lift: 164


Support = 5 
Confidence = 70 
Rules = 456
Rules with high lift: 101


S

In [294]:
#HERE WE CAN SEE THE RULES THAT WE WANT
rules = apriori(baskets, supp=20, zmin=2, target='r', conf=80, 
                report='YXlC') 
print('Number of rules:', len(rules))
#Code that filters all the association rules with a certain lift under
c = 0
for r in rules:
        if r[4]>1: #r[4] indeed refers to index 4 so as we have seen before index 4 refers to refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
            c += 1
            print('%s -- %s' % (str(c), r))
print('Rules with high lift: {}'.format(c))

Number of rules: 19
1 -- ('AMERICAN_Nationality', ('[1289.765, 1703.647)_WarrantyCost_bin',), 83.58543897216273, 22.100214132762314, 1.1353606722827303, 94.89962018448182)
2 -- ('AMERICAN_Nationality', ('CHEVROLET_Make',), 83.58543897216273, 23.70877944325482, 1.1963806283688234, 100.0)
3 -- ('AMERICAN_Nationality', ('OTHER_Auction',), 83.58543897216273, 23.97087794432548, 1.033933462364338, 86.4217823197313)
4 -- ('AMERICAN_Nationality', ('[875.882, 1289.765)_WarrantyCost_bin',), 83.58543897216273, 31.66081370449679, 1.0766907797672676, 89.99567146412726)
5 -- ('AMERICAN_Nationality', ('Alloy_WheelType', 'MANHEIM_Auction'), 83.58543897216273, 30.199571734475374, 1.0121289177771078, 84.59923988881955)
6 -- ('AMERICAN_Nationality', ('Alloy_WheelType',), 83.58543897216273, 51.58201284796574, 1.0195319625367476, 85.21802663478462)
Rules with high lift: 6


In [295]:
# count how many rows for each attribute to make sure i haven't cut anything
for i in df_new:
    conta = 0
    for j in df_new.iterrows():
        conta += 1
    print(i, " = ", conta)

Auction  =  58375
VehicleAge  =  58375
Make  =  58375
Trim  =  58375
Color  =  58375
WheelType  =  58375
Nationality  =  58375
Size  =  58375
VNST  =  58375
VehBCost_bin  =  58375
VehOdo_bin  =  58375
WarrantyCost_bin  =  58375
